# Shoreline Monitor Future Positions

Future Shoreline Positions Vousdoukas 2021

Notebook environment to migrate netcdf files to CF compliant zarr

In [2]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [3]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
from tqdm import tqdm
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# use workaround because : FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\rowe\\AppData\\Local\\miniconda3\\envs\\coclico\\Lib\\site-packages\\coastmonitor\\..\\..\\pyproject.toml'
sys.path.append(r'C:\Users\rowe\Documents\GitHub\coastmonitor\src')
from coastmonitor.io.utils import name_block


# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")



<IPython.core.display.Javascript object>

In [4]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("03_Shorelinemonitor_future")
dataset_dir_shorelinemonitor_fut = dataset_dir.joinpath("shorelinemonitor_future.nc")
dataset_out_file = "ShorelineMonitor_Future"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [5]:
dataset_dir_shorelinemonitor_fut

WindowsPath('P:/1000545-054-globalbeaches/15_GlobalCoastalAtlas/datasets/03_Shorelinemonitor_future/shorelinemonitor_future.nc')

<IPython.core.display.Javascript object>

In [6]:
# # # write csv to netcdf

# # # Load data from CSV file into a pandas dataframe
csv_dir = r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\future_shorelines.csv'
df_fut = pd.read_csv(csv_dir)
df_fut

,transect_id,country_id,continent,country_name,changerate,intercept,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,BOX_028_183_1,CHL,South America,Chile,6.937372,297.416535,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,"[0, 256.6827524645527, 457.8665314232562, 804....","[0, 256.6827524645527, 456.1842314232562, 800....","[0, 256.6827524645527, 457.1225314232562, 802....","[0, 256.6827524645527, 457.68188142325624, 804...","[0, 256.6827524645527, 455.8797314232562, 797....","[0, 256.6827524645527, 456.9316514232562, 801....","[0, 256.6827524645527, 457.6780214232562, 804...."
1,BOX_028_183_90,CHL,South America,Chile,1.080441,464.383193,-74.298876,-50.379169,-74.307625,-50.377870,-74.316375,-50.376571,"[0, 39.976307007810426, 71.30908817609428, 125...","[0, 39.976307007810426, 69.61918817609428, 121...","[0, 39.976307007810426, 70.56508817609428, 123...","[0, 39.976307007810426, 71.12659817609428, 124...","[0, 39.976307007810426, 69.36148817609428, 117...","[0, 39.976307007810426, 70.37420817609427, 122...","[0, 39.976307007810426, 71.11315817609427, 124..."
2,BOX_028_183_228,CHL,South America,Chile,0.103907,611.361480,-74.180387,-50.485299,-74.171445,-50.484748,-74.162504,-50.484197,"[0, 3.8445445286552458, 6.857836186249898, 12....","[0, 3.8445445286552458, -85.91116381375011, -2...","[0, 3.8445445286552458, -37.7821638137501, -99...","[0, 3.8445445286552458, -8.752163813750101, -2...","[0, 3.8445445286552458, -101.6121638137501, -4...","[0, 3.8445445286552458, -49.235163813750106, -...","[0, 3.8445445286552458, -12.370163813750104, -..."
3,BOX_028_183_229,CHL,South America,Chile,0.159166,637.147295,-74.180820,-50.482455,-74.171878,-50.481904,-74.162937,-50.481352,"[0, 5.889157039416696, 10.50498282706762, 18.4...","[0, 5.889157039416696, -83.25001717293237, -20...","[0, 5.889157039416696, -34.13501717293238, -93...","[0, 5.889157039416696, -5.1420171729323805, -1...","[0, 5.889157039416696, -106.97501717293238, -4...","[0, 5.889157039416696, -45.58801717293238, -16...","[0, 5.889157039416696, -7.813017172932382, -36..."
4,BOX_028_183_235,CHL,South America,Chile,2.765432,738.079795,-74.184526,-50.474289,-74.183504,-50.468609,-74.182481,-50.462928,"[0, 102.32099104799883, 182.518524572106, 320....","[0, 102.32099104799883, 89.172524572106, 88.42...","[0, 102.32099104799883, 137.87852457210602, 20...","[0, 102.32099104799883, 166.714524572106, 283....","[0, 102.32099104799883, 72.858524572106, -89.3...","[0, 102.32099104799883, 126.425524572106, 135....","[0, 102.32099104799883, 164.150524572106, 266...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504810,BOX_211_005_33,RUS,Europe,Russia,-0.331204,479.130911,36.771711,64.019196,36.762850,64.019841,36.753988,64.020486,"[0, -12.254560122596638, -21.859485624091302, ...","[0, -12.254560122596638, -93.14248562409131, -...","[0, -12.254560122596638, -56.8694856240913, -1...","[0, -12.254560122596638, -32.987485624091306, ...","[0, -12.254560122596638, -166.05948562409128, ...","[0, -12.254560122596638, -96.4474856240913, -2...","[0, -12.254560122596638, -43.997485624091304, ..."
504811,BOX_211_005_14,RUS,Europe,Russia,-0.035097,470.202918,36.774473,64.058017,36.765792,64.057007,36.757111,64.055996,"[0, -1.2985951551274446, -2.316412979416523, -...","[0, -1.2985951551274446, -69.29441297941652, -...","[0, -1.2985951551274446, -36.42841297941653, -...","[0, -1.2985951551274446, -14.491412979416523, ...","[0, -1.2985951551274446, -137.82641297941652, ...","[0, -1.2985951551274446, -74.99141297941652, -...","[0, -1.2985951551274446, -21.570412979416524, ..."
504812,BOX_211_067_137,RUS,Europe,Russia,-0.052583,349.242539,39.881510,64.701394,39.881905,64.697558,39.882300,64.693722,"[0, -1.9455689669374272, -3.4704743734559513, ...","[0, -1.9455689669374272, -72.24047437345595, -...","[0, -1.9455689669374272, -38.48047437345595, -...","[0, -1.945

<IPython.core.display.Javascript object>

In [7]:
df_fut["RCP4.5_p50"][0]

'[0, 256.6827524645527, 457.1225314232562, 802.8774158348139]'

<IPython.core.display.Javascript object>

In [8]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10

df_final_merged=df_fut

# Filter dataframe within specified range
filtered_df = df_final_merged[
    (df_final_merged["Intersect_lon"] < lon + 18) & 
    (df_final_merged["Intersect_lon"] > lon - 7.5) & 
    (df_final_merged["Intersect_lat"] < lat + 9) & 
    (df_final_merged["Intersect_lat"] > lat - 8)
]

Hazard_map=filtered_df

# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map,
                        lat=Hazard_map["Intersect_lat"],
                        lon=Hazard_map["Intersect_lon"],
                        color="changerate",
                        hover_data="transect_id",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{Hazard_map.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

<IPython.core.display.Javascript object>

In [18]:
df_p=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\01_Hazards\02_Future\Shoreline_Change\Global_Erosion_rate_FC_shore_mon_fut.csv")
df_p

,stations,changerate,continent,country,country_id,intercept,lat,lon,transect_geom,transect_id,Change_rate_SSP1_45_2050,Change_rate_SSP1_45_2100,Change_rate_SSP5_85_2050,Change_rate_SSP5_85_2100
0,0,6.937372,b'South America',b'Chile',b'CHL',297.416535,-50.379144,-74.382469,"b'LINESTRING (-74.382468591 -50.3791437735, -7...",b'BOX_028_183_1',6.911717,6.913856,6.905134,6.898272
1,1,1.080441,b'South America',b'Chile',b'CHL',464.383193,-50.379169,-74.298876,"b'LINESTRING (-74.2988761993 -50.3791687326, -...",b'BOX_028_183_90',1.054786,1.056926,1.048203,1.041341
2,2,0.103907,b'South America',b'Chile',b'CHL',611.361480,-50.485299,-74.180387,"b'LINESTRING (-74.1803866782 -50.4852993327, -...",b'BOX_028_183_228',-1.435404,-1.307106,-1.830335,-2.242043
3,3,0.159166,b'South America',b'Chile',b'CHL',637.147295,-50.482455,-74.180820,"b'LINESTRING (-74.180819763 -50.48245468, -74....",b'BOX_028_183_229',-1.380144,-1.251846,-1.775075,-2.186783
4,4,2.765432,b'South America',b'Chile',b'CHL',738.079795,-50.474289,-74.184526,"b'LINESTRING (-74.1845256308 -50.4742894211, -...",b'BOX_028_183_235',1.226122,1.354420,0.831191,0.419483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504810,504810,-0.331204,b'Europe',b'Russia',b'RUS',479.130911,64.019196,36.771711,"b'LINESTRING (36.771711154 64.0191960871, 36.7...",b'BOX_211_005_33',-1.538446,-1.179090,-2.903204,-2.879939
504811,504811,-0.035097,b'Europe',b'Russia',b'RUS',470.202918,64.058017,36.774473,"b'LINESTRING (36.7744733192 64.0580170063, 36....",b'BOX_211_005_14',-1.211373,-0.861236,-2.541132,-2.518515
504812,504812,-0.052583,b'Europe',b'Russia',b'RUS',349.242539,64.701394,39.881510,"b'LINESTRING (39.8815103752 64.7013937105, 39....",b'BOX_211_067_137',-1.259824,-0.900456,-2.854997,-2.840051
504813,504813,0.155795,b'Europe',b'Russia',b'RUS',367.903231,64.701309,39.877023,"b'LINESTRING (39.8770231358 64.7013093818, 39....",b'BOX_211_067_136',-1.051446,-0.692079,-2.646619,-2.631673


<IPython.core.display.Javascript object>

In [21]:
df_p["transect_id"]=df_p["transect_id"].apply(lambda x : x.replace("b","").replace("'",""))

<IPython.core.display.Javascript object>

In [22]:
df_p

,stations,changerate,continent,country,country_id,intercept,lat,lon,transect_geom,transect_id,Change_rate_SSP1_45_2050,Change_rate_SSP1_45_2100,Change_rate_SSP5_85_2050,Change_rate_SSP5_85_2100
0,0,6.937372,b'South America',b'Chile',b'CHL',297.416535,-50.379144,-74.382469,"b'LINESTRING (-74.382468591 -50.3791437735, -7...",BOX_028_183_1,6.911717,6.913856,6.905134,6.898272
1,1,1.080441,b'South America',b'Chile',b'CHL',464.383193,-50.379169,-74.298876,"b'LINESTRING (-74.2988761993 -50.3791687326, -...",BOX_028_183_90,1.054786,1.056926,1.048203,1.041341
2,2,0.103907,b'South America',b'Chile',b'CHL',611.361480,-50.485299,-74.180387,"b'LINESTRING (-74.1803866782 -50.4852993327, -...",BOX_028_183_228,-1.435404,-1.307106,-1.830335,-2.242043
3,3,0.159166,b'South America',b'Chile',b'CHL',637.147295,-50.482455,-74.180820,"b'LINESTRING (-74.180819763 -50.48245468, -74....",BOX_028_183_229,-1.380144,-1.251846,-1.775075,-2.186783
4,4,2.765432,b'South America',b'Chile',b'CHL',738.079795,-50.474289,-74.184526,"b'LINESTRING (-74.1845256308 -50.4742894211, -...",BOX_028_183_235,1.226122,1.354420,0.831191,0.419483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504810,504810,-0.331204,b'Europe',b'Russia',b'RUS',479.130911,64.019196,36.771711,"b'LINESTRING (36.771711154 64.0191960871, 36.7...",BOX_211_005_33,-1.538446,-1.179090,-2.903204,-2.879939
504811,504811,-0.035097,b'Europe',b'Russia',b'RUS',470.202918,64.058017,36.774473,"b'LINESTRING (36.7744733192 64.0580170063, 36....",BOX_211_005_14,-1.211373,-0.861236,-2.541132,-2.518515
504812,504812,-0.052583,b'Europe',b'Russia',b'RUS',349.242539,64.701394,39.881510,"b'LINESTRING (39.8815103752 64.7013937105, 39....",BOX_211_067_137,-1.259824,-0.900456,-2.854997,-2.840051
504813,504813,0.155795,b'Europe',b'Russia',b'RUS',367.903231,64.701309,39.877023,"b'LINESTRING (39.8770231358 64.7013093818, 39....",BOX_211_067_136,-1.051446,-0.692079,-2.646619,-2.631673


<IPython.core.display.Javascript object>

In [9]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\01_Hazards\02_Future\Shoreline_Change\Global_Erosion_rate_FC_shore_mon_fut_remapped.csv")
df

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,geometry,sampled points,slope,Change_rate_SSP1_45_2050,Change_rate_SSP1_45_2100,Change_rate_SSP5_85_2050,Change_rate_SSP5_85_2100
0,BOX_116_353_2,COL,South America,Colombia,-71.874132,12.257491,-71.879805,12.250684,-71.885477,12.243877,POINT (-71.8798045985 12.250683718),0.0,NaN,NaN,NaN,NaN,NaN
1,BOX_116_353_3,COL,South America,Colombia,-71.869660,12.252394,-71.876981,12.247308,-71.884303,12.242222,POINT (-71.87698143850001 12.247308305),0.0,NaN,NaN,NaN,NaN,NaN
2,BOX_116_353_4,COL,South America,Colombia,-71.866684,12.243923,-71.875632,12.243144,-71.884580,12.242366,POINT (-71.8756318462 12.2431443263),0.0,NaN,NaN,NaN,NaN,NaN
3,BOX_116_353_5,COL,South America,Colombia,-71.866286,12.239551,-71.875233,12.238772,-71.884181,12.237993,POINT (-71.8752334021 12.2387721729),0.0,NaN,NaN,NaN,NaN,NaN
4,BOX_116_353_6,COL,South America,Colombia,-71.865960,12.235759,-71.874835,12.234400,-71.883710,12.233041,POINT (-71.8748349513 12.2343999454),0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37504,BOX_139_002_61,DOM,North America,Dominican Republic,-71.660191,17.969154,-71.668186,17.965259,-71.676182,17.961363,POINT (-71.6681862112 17.9652589019),1.0,NaN,0.255533,0.261496,0.247684,0.245955
37505,BOX_139_002_62,DOM,North America,Dominican Republic,-71.657870,17.964549,-71.666228,17.961418,-71.674587,17.958286,POINT (-71.6662284163 17.9614175026),1.0,NaN,0.132781,0.138744,0.124932,0.123203
37506,BOX_139_002_63,DOM,North America,Dominican Republic,-71.656224,17.960573,-71.664583,17.957442,-71.672941,17.954311,POINT (-71.664582619 17.9574419454),1.0,NaN,0.059195,0.065159,0.051347,0.049618
37507,BOX_139_002_64,DOM,North America,Dominican Republic,-71.654578,17.956598,-71.662937,17.953466,-71.671295,17.950335,POINT (-71.6629367897 17.953466310899998),1.0,NaN,-0.060692,-0.054729,-0.068541,-0.070270


<IPython.core.display.Javascript object>

In [10]:
df["Change_rate_SSP1_45_2050"].isna()

0         True
1         True
2         True
3         True
4         True
         ...  
37504    False
37505    False
37506    False
37507    False
37508    False
Name: Change_rate_SSP1_45_2050, Length: 37509, dtype: bool

<IPython.core.display.Javascript object>

In [11]:
import plotly.express as px

lat = 18.1
lon = -77.3
view_width = 10

df=df.loc[~df["Change_rate_SSP1_45_2050"].isna(),:]

# Plot using Plotly Express
fig = px.scatter_mapbox(df,
                        lat="Intersect_lat",
                        lon="Intersect_lon",
                        color="Change_rate_SSP1_45_2050",
                        range_color=(-2, 0.5),
                        hover_data="transect_id",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
# fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{df_rate.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

# fig_name_shoreline=df_rate.columns[-1]+' shoreline_plot-RAW.png'


<IPython.core.display.Javascript object>

In [12]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged = pd.merge(left=df, right=Hazard_map, left_on="transect_id", right_on="transect_id")
Hazard_map_merged

,transect_id,country_id_x,continent_x,country_name_x,Start_lon_x,Start_lat_x,Intersect_lon_x,Intersect_lat_x,End_lon_x,End_lat_x,...,Intersect_lat_y,End_lon_y,End_lat_y,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,BOX_117_233_9,FRA,Europe,France,-61.177756,14.760801,-61.184958,14.755609,-61.192160,14.750416,...,14.755609,-61.192160,14.750416,"[0, -8.25072866337923, -14.71751599413592, -25...","[0, -8.25072866337923, -16.57631599413592, -30...","[0, -8.25072866337923, -15.680895994135922, -2...","[0, -8.25072866337923, -14.88517599413592, -26...","[0, -8.25072866337923, -17.10861599413592, -35...","[0, -8.25072866337923, -15.908515994135922, -2...","[0, -8.25072866337923, -14.91113599413592, -26..."
1,BOX_117_233_27,FRA,Europe,France,-60.950675,14.741954,-60.941716,14.741317,-60.932757,14.740681,...,14.741317,-60.932757,14.740681,"[0, -2.9500769770308843, -5.262299472541577, -...","[0, -2.9500769770308843, -101.10929947254157, ...","[0, -2.9500769770308843, -58.669299472541574, ...","[0, -2.9500769770308843, -20.218299472541577, ...","[0, -2.9500769770308843, -132.95229947254157, ...","[0, -2.9500769770308843, -71.28829947254157, -...","[0, -2.9500769770308843, -22.73429947254158, -..."
2,BOX_117_233_41,FRA,Europe,France,-60.939809,14.746150,-60.946524,14.751920,-60.953238,14.757691,...,14.751920,-60.953238,14.757691,"[0, 5.928834178856544, 10.575758264987348, 18....","[0, 5.928834178856544, -85.27124173501265, -21...","[0, 5.928834178856544, -42.83124173501265, -10...","[0, 5.928834178856544, -4.3802417350126515, -1...","[0, 5.928834178856544, -117.11424173501265, -4...","[0, 5.928834178856544, -55.450241735012646, -1...","[0, 5.928834178856544, -6.896241735012653, -28..."
3,BOX_117_233_47,FRA,Europe,France,-60.971886,14.744398,-60.963872,14.748324,-60.955858,14.752250,...,14.748324,-60.955858,14.752250,"[0, 13.689296789368857, 24.41874562427958, 42....","[0, 13.689296789368857, -71.42825437572041, -1...","[0, 13.689296789368857, -28.988254375720416, -...","[0, 13.689296789368857, 9.46274562427958, 5.62...","[0, 13.689296789368857, -103.27125437572042, -...","[0, 13.689296789368857, -41.607254375720416, -...","[0, 13.689296789368857, 6.946745624279579, -4...."
4,BOX_117_233_59,FRA,Europe,France,-61.003486,14.784038,-60.996519,14.789520,-60.989552,14.795003,...,14.789520,-60.989552,14.795003,"[0, 8.245790228117565, 14.708706893398901, 25....","[0, 8.245790228117565, -3.2202931066010976, -2...","[0, 8.245790228117565, 5.3880068933989005, 5.0...","[0, 8.245790228117565, 12.321306893398901, 19....","[0, 8.245790228117565, -10.0182931066011, -69....","[0, 8.245790228117565, 3.458706893398901, -6.0...","[0, 8.245790228117565, 12.144406893398902, 19...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9840,BOX_139_002_61,DOM,North America,Dominican Republic,-71.660191,17.969154,-71.668186,17.965259,-71.676182,17.961363,...,17.965259,-71.676182,17.961363,"[0, 10.683854772591845, 19.057686891650317, 33...","[0, 10.683854772591845, 17.188886891650316, 28...","[0, 10.683854772591845, 18.094306891650316, 31...","[0, 10.683854772591845, 18.879926891650317, 33...","[0, 10.683854772591845, 16.685886891650316, 23...","[0, 10.683854772591845, 17.866686891650318, 30...","[0, 10.683854772591845, 18.894076891650318, 33..."
9841,BOX_139_002_62,DOM,North America,Dominican Republic,-71.657870,17.964549,-71.666228,17.961418,-71.674587,17.958286,...,17.961418,-71.674587,17.958286,"[0, 6.142018721459105, 10.95603339503516, 19.2...","[0, 6.142018721459105, 9.10283339503516, 14.94...","[0, 6.142018721459105, 9.992653395035159, 17.1...","[0, 6.142018721459105, 10.76371339503516, 18.8...","[0, 6.142018721459105, 8.53143339503516, 10.42...","[0, 6.142018721459105, 9.765033395035159, 15.8...","[0, 6.142018721459105, 10.76501339503516, 18.6..."
9842,BOX_139_002_63,DOM,North America,Dominican Republic,-71.656224,17.960573,-71.664583,17.957442,-71.672941,17.954311,...,17.957442,-71.672941,17.9

<IPython.core.display.Javascript object>

In [14]:
(Hazard_map_merged["Intersect_lon_x"]==Hazard_map_merged["Intersect_lon_y"]).all()

True

<IPython.core.display.Javascript object>

In [17]:
(Hazard_map_merged["Intersect_lat_x"]==Hazard_map_merged["Intersect_lat_y"]).all()

True

<IPython.core.display.Javascript object>

In [23]:
# Hazard_map["ESL_100yrRP"]=data_points["ESL_RCP0.0_Time_Horizon2000"]
Hazard_map_merged3 = pd.merge(left=Hazard_map_merged, right=df_p, left_on="transect_id", right_on="transect_id")
Hazard_map_merged3

,transect_id,country_id_x,continent_x,country_name_x,Start_lon_x,Start_lat_x,Intersect_lon_x,Intersect_lat_x,End_lon_x,End_lat_x,...,country,country_id,intercept_y,lat,lon,transect_geom,Change_rate_SSP1_45_2050_y,Change_rate_SSP1_45_2100_y,Change_rate_SSP5_85_2050_y,Change_rate_SSP5_85_2100_y
0,BOX_117_233_9,FRA,Europe,France,-61.177756,14.760801,-61.184958,14.755609,-61.192160,14.750416,...,b'France',b'FRA',908.649342,14.760801,-61.177756,b'LINESTRING (-61.1777562853 14.76080135630000...,-0.256213,-0.250250,-0.264062,-0.265790
1,BOX_117_233_27,FRA,Europe,France,-60.950675,14.741954,-60.941716,14.741317,-60.932757,14.740681,...,b'France',b'FRA',745.795543,14.741954,-60.950675,"b'LINESTRING (-60.9506753977 14.7419536281, -6...",-1.921352,-1.590744,-2.356490,-2.452263
2,BOX_117_233_41,FRA,Europe,France,-60.939809,14.746150,-60.946524,14.751920,-60.953238,14.757691,...,b'France',b'FRA',1119.474140,14.746150,-60.939809,"b'LINESTRING (-60.9398085764 14.7461498959, -6...",-1.681382,-1.350774,-2.116520,-2.212293
3,BOX_117_233_47,FRA,Europe,France,-60.971886,14.744398,-60.963872,14.748324,-60.955858,14.752250,...,b'France',b'FRA',889.990242,14.744398,-60.971886,b'LINESTRING (-60.9718855034 14.74439784839999...,-1.471640,-1.141032,-1.906778,-2.002551
4,BOX_117_233_59,FRA,Europe,France,-61.003486,14.784038,-60.996519,14.789520,-60.989552,14.795003,...,b'France',b'FRA',821.384663,14.784038,-61.003486,b'LINESTRING (-61.00348595810001 14.7840377382...,-0.098544,-0.040850,-0.165072,-0.181407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14024,BOX_139_002_61,DOM,North America,Dominican Republic,-71.660191,17.969154,-71.668186,17.965259,-71.676182,17.961363,...,b'Dominican Republic',b'DOM',892.636242,17.969154,-71.660191,"b'LINESTRING (-71.6601908294 17.9691544308, -7...",0.255533,0.261496,0.247684,0.245955
14025,BOX_139_002_62,DOM,North America,Dominican Republic,-71.657870,17.964549,-71.666228,17.961418,-71.674587,17.958286,...,b'Dominican Republic',b'DOM',887.783993,17.964549,-71.657870,"b'LINESTRING (-71.6578700793 17.9645487929, -7...",0.132781,0.138744,0.124932,0.123203
14026,BOX_139_002_63,DOM,North America,Dominican Republic,-71.656224,17.960573,-71.664583,17.957442,-71.672941,17.954311,...,b'Dominican Republic',b'DOM',931.399153,17.960573,-71.656224,"b'LINESTRING (-71.6562242238 17.9605733279, -7...",0.059195,0.065159,0.051347,0.049618
14027,BOX_139_002_64,DOM,North America,Dominican Republic,-71.654578,17.956598,-71.662937,17.953466,-71.671295,17.950335,...,b'Dominican Republic',b'DOM',1061.648069,17.956598,-71.654578,"b'LINESTRING (-71.6545783683 17.9565977736, -7...",-0.060692,-0.054729,-0.068541,-0.070270


<IPython.core.display.Javascript object>

In [24]:
Hazard_map_merged3.columns

Index(['transect_id', 'country_id_x', 'continent_x', 'country_name_x',
       'Start_lon_x', 'Start_lat_x', 'Intersect_lon_x', 'Intersect_lat_x',
       'End_lon_x', 'End_lat_x', 'geometry', 'sampled points', 'slope',
       'Change_rate_SSP1_45_2050_x', 'Change_rate_SSP1_45_2100_x',
       'Change_rate_SSP5_85_2050_x', 'Change_rate_SSP5_85_2100_x',
       'country_id_y', 'continent_y', 'country_name_y', 'changerate_x',
       'intercept_x', 'Start_lon_y', 'Start_lat_y', 'Intersect_lon_y',
       'Intersect_lat_y', 'End_lon_y', 'End_lat_y', 'Ambient', 'RCP4.5_p5',
       'RCP4.5_p50', 'RCP4.5_p95', 'RCP8.5_p5', 'RCP8.5_p50', 'RCP8.5_p95',
       'stations', 'changerate_y', 'continent', 'country', 'country_id',
       'intercept_y', 'lat', 'lon', 'transect_geom',
       'Change_rate_SSP1_45_2050_y', 'Change_rate_SSP1_45_2100_y',
       'Change_rate_SSP5_85_2050_y', 'Change_rate_SSP5_85_2100_y'],
      dtype='object')

<IPython.core.display.Javascript object>

In [43]:
(Hazard_map_merged3["lat"]==Hazard_map_merged3["Start_lat_y"]).all()

True

<IPython.core.display.Javascript object>

# This shows that there is some mismatch between the transect_IDs when they became zarrs

In [41]:
Hazard_map_merged3[["transect_id",'lat', 'Start_lat_x', 'Start_lat_y']]

,transect_id,lat,Start_lat_x,Start_lat_y
0,BOX_117_233_9,14.760801,14.760801,14.760801
1,BOX_117_233_27,14.741954,14.741954,14.741954
2,BOX_117_233_41,14.746150,14.746150,14.746150
3,BOX_117_233_47,14.744398,14.744398,14.744398
4,BOX_117_233_59,14.784038,14.784038,14.784038
...,...,...,...,...
14024,BOX_139_002_61,17.969154,17.969154,17.969154
14025,BOX_139_002_62,17.964549,17.964549,17.964549
14026,BOX_139_002_63,17.960573,17.960573,17.960573
14027,BOX_139_002_64,17.956598,17.956598,17.956598


<IPython.core.display.Javascript object>

In [37]:
df_p.loc[df_p["transect_id"]=="BOX_117_233_9"]

,stations,changerate,continent,country,country_id,intercept,lat,lon,transect_geom,transect_id,Change_rate_SSP1_45_2050,Change_rate_SSP1_45_2100,Change_rate_SSP5_85_2050,Change_rate_SSP5_85_2100
144986,144986,-0.222993,b'Europe',b'France',b'FRA',908.649342,14.760801,-61.177756,b'LINESTRING (-61.1777562853 14.76080135630000...,BOX_117_233_9,-0.256213,-0.25025,-0.264062,-0.26579


<IPython.core.display.Javascript object>

In [38]:
Hazard_map_merged3.loc[Hazard_map_merged3['Intersect_lat_x']==14.760801]

,transect_id,country_id_x,continent_x,country_name_x,Start_lon_x,Start_lat_x,Intersect_lon_x,Intersect_lat_x,End_lon_x,End_lat_x,...,country,country_id,intercept_y,lat,lon,transect_geom,Change_rate_SSP1_45_2050_y,Change_rate_SSP1_45_2100_y,Change_rate_SSP5_85_2050_y,Change_rate_SSP5_85_2100_y


<IPython.core.display.Javascript object>

In [33]:

filtered_df = Hazard_map_merged3.loc[
    Hazard_map_merged3[['lat', 'Intersect_lat_x', 'Intersect_lat_y']].apply(lambda x: x.isin(x)).any(axis=1)
]
filtered_df

,transect_id,country_id_x,continent_x,country_name_x,Start_lon_x,Start_lat_x,Intersect_lon_x,Intersect_lat_x,End_lon_x,End_lat_x,...,country,country_id,intercept_y,lat,lon,transect_geom,Change_rate_SSP1_45_2050_y,Change_rate_SSP1_45_2100_y,Change_rate_SSP5_85_2050_y,Change_rate_SSP5_85_2100_y
0,BOX_117_233_9,FRA,Europe,France,-61.177756,14.760801,-61.184958,14.755609,-61.192160,14.750416,...,b'France',b'FRA',908.649342,14.760801,-61.177756,b'LINESTRING (-61.1777562853 14.76080135630000...,-0.256213,-0.250250,-0.264062,-0.265790
1,BOX_117_233_27,FRA,Europe,France,-60.950675,14.741954,-60.941716,14.741317,-60.932757,14.740681,...,b'France',b'FRA',745.795543,14.741954,-60.950675,"b'LINESTRING (-60.9506753977 14.7419536281, -6...",-1.921352,-1.590744,-2.356490,-2.452263
2,BOX_117_233_41,FRA,Europe,France,-60.939809,14.746150,-60.946524,14.751920,-60.953238,14.757691,...,b'France',b'FRA',1119.474140,14.746150,-60.939809,"b'LINESTRING (-60.9398085764 14.7461498959, -6...",-1.681382,-1.350774,-2.116520,-2.212293
3,BOX_117_233_47,FRA,Europe,France,-60.971886,14.744398,-60.963872,14.748324,-60.955858,14.752250,...,b'France',b'FRA',889.990242,14.744398,-60.971886,b'LINESTRING (-60.9718855034 14.74439784839999...,-1.471640,-1.141032,-1.906778,-2.002551
4,BOX_117_233_59,FRA,Europe,France,-61.003486,14.784038,-60.996519,14.789520,-60.989552,14.795003,...,b'France',b'FRA',821.384663,14.784038,-61.003486,b'LINESTRING (-61.00348595810001 14.7840377382...,-0.098544,-0.040850,-0.165072,-0.181407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14024,BOX_139_002_61,DOM,North America,Dominican Republic,-71.660191,17.969154,-71.668186,17.965259,-71.676182,17.961363,...,b'Dominican Republic',b'DOM',892.636242,17.969154,-71.660191,"b'LINESTRING (-71.6601908294 17.9691544308, -7...",0.255533,0.261496,0.247684,0.245955
14025,BOX_139_002_62,DOM,North America,Dominican Republic,-71.657870,17.964549,-71.666228,17.961418,-71.674587,17.958286,...,b'Dominican Republic',b'DOM',887.783993,17.964549,-71.657870,"b'LINESTRING (-71.6578700793 17.9645487929, -7...",0.132781,0.138744,0.124932,0.123203
14026,BOX_139_002_63,DOM,North America,Dominican Republic,-71.656224,17.960573,-71.664583,17.957442,-71.672941,17.954311,...,b'Dominican Republic',b'DOM',931.399153,17.960573,-71.656224,"b'LINESTRING (-71.6562242238 17.9605733279, -7...",0.059195,0.065159,0.051347,0.049618
14027,BOX_139_002_64,DOM,North America,Dominican Republic,-71.654578,17.956598,-71.662937,17.953466,-71.671295,17.950335,...,b'Dominican Republic',b'DOM',1061.648069,17.956598,-71.654578,"b'LINESTRING (-71.6545783683 17.9565977736, -7...",-0.060692,-0.054729,-0.068541,-0.070270


<IPython.core.display.Javascript object>

In [ ]:
# Plot using Plotly Express
fig = px.scatter_mapbox(Hazard_map_merged,
                        lat="Intersect_lat_y",
                        lon="Intersect_lon_x",
                        color="transect_id",
                        # range_color=(-2, 0.5),
                        hover_data="transect_id",
                        mapbox_style="carto-positron",width=2000,
                        height=1200)

# Update map layout to adjust center and zoom
# fig.update_layout(mapbox=dict(center=dict(lat=lat+1, lon=lon+4), zoom=5.5),title=f"{df_rate.columns[-1]} Distribution along Caribbean's shoreline")
fig.show()

# fig_name_shoreline=df_rate.columns[-1]+' shoreline_plot-RAW.png'